# <font color='red'><b>Bootstrap assignment</b> </font>

<font color='blue'> <b>Importing packages</b> </font>

In [12]:
import numpy as np # importing numpy for numerical computation
from sklearn.datasets import load_boston # here we are using sklearn's boston dataset
from sklearn.metrics import mean_squared_error # importing mean_squared_error metric
from sklearn.tree import DecisionTreeRegressor as dtr

import warnings
warnings.filterwarnings("ignore")

In [13]:
boston = load_boston()
x=boston.data #independent variables
y=boston.target #target variable

In [14]:
x.shape

(506, 13)

# <font color='blue'> <b>Task - 1</b></font>

<font color='blue'><b>Step - 1</b></font>

*  <font color='blue'> <b>Creating samples</b></font>

In [15]:
def generating_samples(input_data, target_data):
    '''In this function, we will write code for generating 30 samples '''

    selecting_rows = np.random.choice(506,replace=False,size = 303)
    replacing_rows = np.random.choice(selecting_rows,replace = True,size =203)
    no_of_cols = np.random.choice(np.arange(1,14))
    select_cols = np.random.choice(13,replace=False,size= no_of_cols)

    sample_data = input_data[selecting_rows[:,None],select_cols]
    target_of_sample_data = target_data[selecting_rows]

    # Replicating data
    rep_sample_data = input_data[replacing_rows[:,None],select_cols]
    target_of_rep_sample_data = target_data[replacing_rows]

    #Concatinating data
    final_sample_data = np.vstack((sample_data,rep_sample_data))
    final_target_data = np.vstack((target_of_sample_data.reshape(-1,1),target_of_rep_sample_data.reshape(-1,1)))

    return final_sample_data,final_target_data,selecting_rows,select_cols




    
    

<font color='cyan'> <b> Grader function - 1 </b> </fongt>

In [16]:
def grader_samples(a,b,c,d):
    length = (len(a)==506  and len(b)==506)
    sampled = (len(a)-len(set([str(i) for i in a]))==203)
    rows_length = (len(c)==303)
    column_length= (len(d)>=3)
    assert(length and sampled and rows_length and column_length)
    return True
a,b,c,d = generating_samples(x, y)
grader_samples(a,b,c,d)

True

*  <font color='blue'> <b>Create 30 samples </b>

In [17]:
# Use generating_samples function to create 30 samples 
# store these created samples in a list
list_input_data =[]
list_output_data =[]
list_selected_rows= []
list_selected_columns=[]
for i in range(30):
    a,b,c,d = generating_samples(x,y)
    list_input_data.append(a)
    list_output_data.append(b)
    list_selected_rows.append(c)
    list_selected_columns.append(d)

<font color='cyan'> <b>Grader function - 2 </b></font>

In [18]:
def grader_30(a):
    assert(len(a)==30 and len(a[0])==506)
    return True
grader_30(list_input_data)

True

<br>

<font color='red'><b>Step - 2 </b></font>

*  <font color='blue'><b>Building regression trees</b></font>

In [19]:
list_of_models = []
for i in range(30):
    model = dtr()
    x_data = list_input_data[i]
    y_data = list_output_data[i]
    model.fit(x_data,y_data)
    list_of_models.append(model)


After getting predicted_y for each data point, we can use sklearns mean_squared_error to calculate the MSE between predicted_y and actual_y.

*  <font color='blue'><b>calculating MSE</b></font>

In [20]:
predict_y_of_data_point = []
for data_point in x:
    array_of_y = []
    for idx, model in enumerate(list_of_models):
        array_of_y.append(model.predict(data_point[list_selected_columns[idx]].reshape(1,-1)))
    predict_y_of_data_point.append(np.median(array_of_y))

In [21]:
task1_mse = mean_squared_error(y,predict_y_of_data_point)
print(task1_mse)

1.9952123475606867


<font color='blue'><b>Step - 3 </b></font>

Now calculate the $OOB Score =  \frac{1}{506}\sum_{i=1}^{506}(y^{i} - y^{i}_{pred})^{2}$.

*  <font color='blue'><b>calculating OOB score </b></font>

In [23]:
y_pred =  []
for i,data_point in zip(range(506),x):
    y_pred_data_point = []
    for model,columns,rows in zip(list_of_models,list_selected_columns,list_selected_rows):
        if i not in rows:
            y_pred_data_point.append(model.predict(data_point[columns.reshape(1,-1)]))
    y_pred.append(np.median(y_pred_data_point))

In [24]:
task1_OOBScore = mean_squared_error(y,y_pred)
task1_OOBScore

20.28944347640152

# <font color='blue'><b>Task 2</b></font>

In [26]:
mse_list = []
oobscore_list = []

for i in range(35):
    #making samples
    list_input_data =[]
    list_output_data =[]
    list_selected_rows= []
    list_selected_columns=[]
    for i in range(30):
        a,b,c,d = generating_samples(x,y)
        list_input_data.append(a)
        list_output_data.append(b)
        list_selected_rows.append(c)
        list_selected_columns.append(d)

    #creating decision trees
    list_of_models = []
    for i in range(30):
        model = dtr()
        x_data = list_input_data[i]
        y_data = list_output_data[i]
        model.fit(x_data,y_data)
        list_of_models.append(model)
    
    #mse of train points
    predict_y_of_data_point = []
    for data_point in x:
        array_of_y = []
        for idx, model in enumerate(list_of_models):
            array_of_y.append(model.predict(data_point[list_selected_columns[idx]].reshape(1,-1)))
        predict_y_of_data_point.append(np.median(array_of_y))
    mse = mean_squared_error(y,predict_y_of_data_point)
    mse_list.append(mse)

    #oobscore 
    y_pred =  []
    for i,data_point in zip(range(506),x):
        y_pred_data_point = []
        for model,columns,rows in zip(list_of_models,list_selected_columns,list_selected_rows):
            if i not in rows:
                y_pred_data_point.append(model.predict(data_point[columns.reshape(1,-1)]))
        y_pred.append(np.median(y_pred_data_point))
    OOBScore = mean_squared_error(y,y_pred)
    oobscore_list.append(OOBScore)


In [27]:
def conf_int(var):
    m = np.mean(var)
    std = np.std(var)
    n = len(var)
    left  = np.round(m - 2*(std/np.sqrt(n)), 3) 
    right = np.round(m + 2*(std/np.sqrt(n)), 3)
    return left,right

    

In [28]:
print("confidence interval  of MSE: ",conf_int(mse_list))
print("confidence interval  of OOBScore: ",conf_int(oobscore_list))

confidence interval  of MSE:  (0.143, 0.234)
confidence interval  of OOBScore:  (15.513, 17.205)


# <font color='blue'><b>Task 3</b></font>

In [29]:
xq= np.array([0.18,20.0,5.00,0.0,0.421,5.60,72.2,7.95,7.0,30.0,19.1,372.13,18.60])

In [30]:
array_y = []
for idx,model in enumerate(list_of_models):
    array_y.append(model.predict(xq[list_selected_columns[idx]].reshape(1,-1)))
print("yq of Xq: ",np.median(array_y))

yq of Xq:  19.1


<br><br><br>

#Task1 Observations:


In [31]:
print("Mean square error\t",task1_mse)
print("OOBScore         \t",task1_OOBScore)

Mean square error	 1.9952123475606867
OOBScore         	 20.28944347640152


#Task2 Observations

In [32]:
print("confidence interval  of MSE:     \t",conf_int(mse_list))
print("confidence interval  of OOBScore:\t",conf_int(oobscore_list))

confidence interval  of MSE:     	 (0.143, 0.234)
confidence interval  of OOBScore:	 (15.513, 17.205)


#Task3 Observations

In [33]:
print("Xq:[0.18,20.0,5.00,0.0,0.421,5.60,72.2,7.95,7.0,30.0,19.1,372.13,18.60]\nYq ",np.median(array_y))

Xq:[0.18,20.0,5.00,0.0,0.421,5.60,72.2,7.95,7.0,30.0,19.1,372.13,18.60]
Yq  19.1
